In [7]:
import gym
import numpy as np

In [8]:
env = gym.make("LunarLander-v2")
obs = env.reset(seed=20, return_info=False)

In [9]:
obs

array([-0.00316305,  1.3999956 , -0.3203935 , -0.48554128,  0.00367194,
        0.072574  ,  0.        ,  0.        ], dtype=float32)

In [10]:
3.14/70

0.04485714285714286

In [19]:
LEARNING_RATE = 0.15

DISCOUNT = 0.995
EPISODES = 50
total = 0
total_reward = 0
prior_reward = 0

Observation = [15, 15, 10, 10, 35, 10, 2, 2]


np_array_win_size = np.array([1.5,1.5,5,5,3.14,5,1,1])*2/(np.array(Observation)-1)

epsilon = 1 # Taux d'apprentissage

epsilon_decay_value = 0.99995

In [20]:
np_array_win_size

array([0.21428571, 0.21428571, 1.11111111, 1.11111111, 0.18470588,
       1.11111111, 2.        , 2.        ])

In [27]:
def get_discrete_state(state):
    discrete_state = state/np_array_win_size+ np.array([15,15,10,10, 35, 10,0,0])-1
    return tuple(discrete_state.astype(int))

In [28]:
get_discrete_state([1.5,1.5,5,5,3.14,5,1,1])

(21, 21, 13, 13, 51, 13, 0, 0)

In [29]:
q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
q_table.shape

(15, 15, 10, 10, 35, 10, 2, 2, 4)

In [30]:
import time
import math
rewards = []
for episode in range(EPISODES + 1): #go through the episodes
    t0 = time.time() #set the initial time
    discrete_state = get_discrete_state(env.reset()) # get the discrete start for the restarted environment 
    done = False
    episode_reward = 0 # reward starts as 0 for each episode

    if episode % 2000 == 0: 
        print("Episode: " + str(episode))

    while not done: 

        if np.random.random() > epsilon:

            action = np.argmax(q_table[discrete_state]) #take cordinated action
        else:

            action = np.random.randint(0, env.action_space.n) # do a random action

        new_state, reward, done, _ = env.step(action) # step action to get new states, reward, and the "done" status.

        episode_reward += reward # add the reward

        new_discrete_state = get_discrete_state(new_state)

        if episode % 2000 == 0: # render
            env.render()

        if not done: # update q-table
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    if epsilon > 0.01: # epsilon modification
       # if episode_reward > prior_reward and episode > 10000: 
        if episode > 10000:
            epsilon *= 1 - math.pow(5, -6);
            #epsilon = math.pow(epsilon_decay_value, episode - 10000)

            if episode % 500 == 0:
                print("Epsilon: " + str(epsilon))

    t1 = time.time() # episode has finished
    episode_total = t1 - t0 # episode total time
    total = total + episode_total

    total_reward += episode_reward # episode total reward
    prior_reward = episode_reward
    rewards.append(episode_reward)
    if episode % 1000 == 0: # every 1000 episodes print the average time and the average reward
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

env.close()

Episode: 0


error: display Surface quit

In [17]:
env.close()